In [ ]:
##
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import sys
from scipy.optimize import curve_fit

In [ ]:
##
#Define Path to Code database
DirPath = '/Your/Path/To/Code/'

In [ ]:
##
#Load functions
sys.path.append(''.join([DirPath,'bin']))
from EPGMotion import *
from MotionSimulation import *
from EPGForwardMotionModel import *
from Jacobian_MotionCorrection import *
from ParameterOptionsExperimental import *

In [ ]:
##
#Choose Direction ('LR', 'AP', 'SI')
Dir = 'AP'

##
#Choose Region ('Thalamus', 'Callosum' or 'CSF')
Region = 'Thalamus'

##
#Define Precision (Smaller number leads to more accurate fitting, but is slower)
Precision = 1E-4

In [ ]:
##
#Read Parameters
opt=ParameterOptionsExperimental()

In [ ]:
##
#Conventional EPG simulates a maximum k-value pathway equal to the number of TRs. This piece of code identifies where increasing the k-value leads to no meaningful change in the signal, with subsequent thresholding to accelerate modelling
opt["kOrder"] = kOrder(opt.copy())

In [ ]:
##
#Load DW-SSFP Timeseries Data
Magnitude = np.genfromtxt(''.join([DirPath,'/Data/Magnitude_',Region,'_',Dir,'.csv']), delimiter=",")
Phase = np.genfromtxt(''.join([DirPath,'/Data/Phase_',Region,'_',Dir,'.csv']), delimiter=",")

In [ ]:
##
#Convert data to complex and take transpose
Comp = np.transpose(Magnitude*np.exp(1j*Phase))

In [ ]:
##
#Define lower and upper parameter bounds (fix S0 equal to 1, define MotionParameters between -5 and 5 mm/s))
low = [1E-6, 0, -20*np.pi, -20*np.pi, *np.ones((int(opt["nTR"])-int(opt["SteadyStateTR"])))*-5, *np.ones((int(opt["nTR"])-int(opt["SteadyStateTR"])))*-5]
high = [10E-3, 1000, 20*np.pi, 20*np.pi, *np.ones((int(opt["nTR"])-int(opt["SteadyStateTR"])))*5, *np.ones((int(opt["nTR"])-int(opt["SteadyStateTR"])))*5]

In [ ]:
##
#Fitting without Motion Information

##
#Initialise fitting parameters (D, S0, Phi)
par_init = [0.5E-4, 1, np.pi/2, np.pi/2]

##
#Initialise Array
poptnoMotion = np.zeros((4,*Comp.shape[1:]))

##
#Define Input Data (1D array: Real & Imaginary Components - Fitting to After Dummy Region)
Data = np.concatenate((np.real(Comp).reshape(np.prod(Comp.shape)),np.imag(Comp).reshape(np.prod(Comp.shape))))

##
#Perform Fitting
poptnoMotion, pcov, infodict, mesg, ier  = curve_fit(lambda x, *theta: EPGForwardModelFitting(x, theta, opt.copy()), 1, Data, p0=par_init, method='trf',absolute_sigma=False,bounds=(low[0:4],high[0:4]),verbose=1,jac=lambda x, *theta: EPGForwardModelFittingJacobian(x,theta,opt.copy()), x_scale='jac',full_output=True,tr_solver='exact',max_nfev=1E5,ftol=Precision, xtol=Precision, gtol=Precision)


In [ ]:
##
#Fitting with Motion Information

##
#Initialise fitting parameters (D, S0, Phi, MotionVector)
par_init = [0.5E-4, 1, np.pi/2, np.pi/2, *np.zeros(int(opt["nTR"])-int(opt["SteadyStateTR"])), *np.zeros(int(opt["nTR"])-int(opt["SteadyStateTR"]))]

##
#Initialise Array
poptMotion = np.zeros((len(low),*Comp.shape[1:]))

##
#Define Input Data (1D array: Real & Imaginary Components - Fitting to After Dummy Region)
Data = np.concatenate((np.real(Comp).reshape(np.prod(Comp.shape)),np.imag(Comp).reshape(np.prod(Comp.shape))))

##
#Perform Fitting
poptMotion, pcov, infodict, mesg, ier  = curve_fit(lambda x, *theta: EPGForwardMotionModelFitting(x, theta, opt.copy()), 1, Data, p0=par_init, method='trf',absolute_sigma=False,bounds=(low,high),verbose=1,jac=lambda x, *theta: EPGForwardMotionModelFittingJacobian(x,theta,opt.copy()), x_scale='jac',full_output=True,tr_solver='exact',max_nfev=1E5,ftol=Precision, xtol=Precision, gtol=Precision)

In [ ]:
##
#Reconstruct Signal & Motion Profile

#Create Reconstructed Motion Array
TRecon = np.zeros([int(opt['nTR']),2], dtype = 'f8')
TRecon[-(int(opt["nTR"])-int(opt["SteadyStateTR"])):,0] = poptMotion[4:4+Magnitude.shape[1]+(int(opt["nDummy"])-int(opt["SteadyStateTR"]))]
TRecon[-(int(opt["nTR"])-int(opt["SteadyStateTR"])):,1] = poptMotion[4+Magnitude.shape[1]+(int(opt["nDummy"])-int(opt["SteadyStateTR"])):]


#Initialise Signal Array
SignalRecon = np.zeros_like(TRecon, dtype = 'c8')

##
#Reconstruct Signal
for k in range(TRecon.shape[1]):
    ##
    #Declare outputs from fitting to the options file 
    optRecon = opt.copy()
    optRecon['D'] = np.asarray([poptMotion[0]], dtype='f8')   
    optRecon['phi'] = np.asarray([poptMotion[2+k]], dtype='f8') 
    optRecon["G"] = np.asarray([opt["GArr"][k]], dtype='f8')
    ##
    #Reconstruct signal
    SignalRecon[:,k] = poptMotion[1]*EPGMotion(optRecon.copy(), TRecon[:,k])*np.exp(1j*optRecon['phi'])

In [ ]:
##
#Print statement on diffusion coefficients
print(''.join(['Without Motion correction, estimated diffusion coefficient D = ',str(poptnoMotion[0]*10**3),' x 10^3 mm^2/s']))
print(''.join(['With Motion correction, estimated diffusion coefficient D = ',str(poptMotion[0]*10**3),' x 10^3 mm^2/s']))

In [ ]:
##
#Plot b0 Data
fig, axs = plt.subplots(3, 1)
fig.set_size_inches(8,12)
#Define x-axis
Time = range(SignalRecon.shape[0])*opt["TR"]/1E3
#Plot Magnitude data
axs[0].plot(Time[int(opt['nDummy']):],np.abs(Comp[:,0]),'#1f77b4',linewidth=2,label = 'Data (Monte Carlo)')
axs[0].plot(Time,np.abs(SignalRecon[:,0]),'#ff7f0e',linewidth=2,linestyle='--',label = 'Fit (EPG + Motion)' )
axs[0].axvspan(Time[int(opt["SteadyStateTR"])],Time[int(opt["nDummy"])], alpha=0.1,color='#d62728',label = 'Dummy Measurements')
axs[0].axvspan(Time[int(opt["nDummy"])],Time[-1],alpha=0.1,color='#2ca02c',label = 'Measured Data')
#Plot Phase data (multiply by -i to characterise the motion-free phase as 0)
axs[1].plot(Time[int(opt['nDummy']):],np.angle(Comp[:,0]),'#1f77b4',linewidth=2)
axs[1].plot(Time,np.angle(SignalRecon[:,0]),'#ff7f0e',linewidth=2,linestyle='--')
axs[1].axvspan(Time[int(opt["SteadyStateTR"])],Time[int(opt["nDummy"])], alpha=0.1,color='#d62728')
axs[1].axvspan(Time[int(opt["nDummy"])],Time[-1],alpha=0.1,color='#2ca02c')
#Plot Time-Series Data
axs[2].plot(Time,TRecon[:,0],'#1f77b4',linewidth=2)
axs[2].axvspan(Time[int(opt["SteadyStateTR"])],Time[int(opt["nDummy"])], alpha=0.1,color='#d62728')
axs[2].axvspan(Time[int(opt["nDummy"])],Time[-1],alpha=0.1,color='#2ca02c')
#Add labels etc
fig.subplots_adjust(hspace=0.4)
axs[0].text(-0.1, 1.05, '(a)', transform=axs[0].transAxes, size=20)
axs[2].text(-0.1, 1.05, '(b)', transform=axs[2].transAxes, size=20)
axs[0].set_xlim([0,Time[-1]])
axs[1].set_xlim([0,Time[-1]])
axs[2].set_xlim([0,Time[-1]])
axs[1].set_ylim([-np.pi,np.pi])
axs[0].set_ylabel('Amplitude (a.u.)',fontsize=12)
axs[1].set_ylabel('Angle (rad.)',fontsize=12)
axs[2].set_ylabel('Velocity (mm/s)',fontsize=12)
axs[0].set_xlabel('Time (s)',fontsize=12)
axs[1].set_xlabel('Time (s)',fontsize=12)
axs[2].set_xlabel('Time (s)',fontsize=12)
axs[0].set_title(r'Magnitude (b$_0$)',fontsize=16)
axs[1].set_title(r'Phase (b$_0$)',fontsize=16)
axs[2].set_title(r'$T(t)$',fontsize=16)
axs[1].set_yticks([-np.pi,0, np.pi],[r'-$\pi$',0, r'$\pi$'])
axs[0].legend(fontsize=10)

In [ ]:
##
#Plot b500 Data
fig, axs = plt.subplots(3, 1)
fig.set_size_inches(8,12)
#Define x-axis
Time = range(SignalRecon.shape[0])*opt["TR"]/1E3
#Plot Magnitude data
axs[0].plot(Time[int(opt['nDummy']):],np.abs(Comp[:,1]),'#1f77b4',linewidth=2,label = 'Data (Monte Carlo)')
axs[0].plot(Time,np.abs(SignalRecon[:,1]),'#ff7f0e',linewidth=2,linestyle='--',label = 'Fit (EPG + Motion)' )
axs[0].axvspan(Time[int(opt["SteadyStateTR"])],Time[int(opt["nDummy"])], alpha=0.1,color='#d62728',label = 'Dummy Measurements')
axs[0].axvspan(Time[int(opt["nDummy"])],Time[-1],alpha=0.1,color='#2ca02c',label = 'Measured Data')
#Plot Phase data (multiply by -i to characterise the motion-free phase as 0)
axs[1].plot(Time[int(opt['nDummy']):],np.angle(Comp[:,1]),'#1f77b4',linewidth=2)
axs[1].plot(Time,np.angle(SignalRecon[:,1]),'#ff7f0e',linewidth=2,linestyle='--')
axs[1].axvspan(Time[int(opt["SteadyStateTR"])],Time[int(opt["nDummy"])], alpha=0.1,color='#d62728')
axs[1].axvspan(Time[int(opt["nDummy"])],Time[-1],alpha=0.1,color='#2ca02c')
#Plot Time-Series Data
axs[2].plot(Time,TRecon[:,1],'#1f77b4',linewidth=2)
axs[2].axvspan(Time[int(opt["SteadyStateTR"])],Time[int(opt["nDummy"])], alpha=0.1,color='#d62728')
axs[2].axvspan(Time[int(opt["nDummy"])],Time[-1],alpha=0.1,color='#2ca02c')
#Add labels etc
fig.subplots_adjust(hspace=0.4)
axs[0].text(-0.1, 1.05, '(a)', transform=axs[0].transAxes, size=20)
axs[2].text(-0.1, 1.05, '(b)', transform=axs[2].transAxes, size=20)
axs[0].set_xlim([0,Time[-1]])
axs[1].set_xlim([0,Time[-1]])
axs[2].set_xlim([0,Time[-1]])
axs[1].set_ylim([-np.pi,np.pi])
axs[0].set_ylabel('Amplitude (a.u.)',fontsize=12)
axs[1].set_ylabel('Angle (rad.)',fontsize=12)
axs[2].set_ylabel('Velocity (mm/s)',fontsize=12)
axs[0].set_xlabel('Time (s)',fontsize=12)
axs[1].set_xlabel('Time (s)',fontsize=12)
axs[2].set_xlabel('Time (s)',fontsize=12)
axs[0].set_title(r'Magnitude (b$_{500}$)',fontsize=16)
axs[1].set_title(r'Phase (b$_{500}$)',fontsize=16)
axs[2].set_title(r'$T(t)$',fontsize=16)
axs[1].set_yticks([-np.pi,0, np.pi],[r'-$\pi$',0, r'$\pi$'])
axs[0].legend(fontsize=10)